## POKEMON COMPETITION

## Repository cloning
All our auxiliary code (feature engineering, utility functions, and model definitions) is stored in a separate GitHub repository.
We clone the repository directly into the Kaggle environment so that the notebook can import and use the modules inside the src/ folder.

In [2]:
!git clone https://github.com/maf1013/pokemonprediction-fds.git


Cloning into 'pokemonprediction-fds'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 18 (delta 4), reused 18 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (18/18), 17.37 KiB | 889.00 KiB/s, done.
Resolving deltas: 100% (4/4), done.


### Add path to `sys.path`

After cloning the repository, we add its folder to `sys.path` so that Python can find the modules in `src` and import them without errors.


In [7]:
import sys

repo_root = "/kaggle/working/pokemonprediction-fds"
src_path  = "/kaggle/working/pokemonprediction-fds/src"


for p in [repo_root, src_path]:
    if p not in sys.path:
        sys.path.append(p)


### Features (`src/features.py`)

This project uses a custom module `features.py` to transform each battle into a set of numerical variables that the models can learn from. The main functions are:

#### learn_type_effectiveness()

Automatically learns a type effectiveness table using only the real outcomes from the dataset.  
For each pair of types:

- Counts how many times they faced each other.  
- Calculates how many times each one won.  
- If there is little data → it is considered neutral (1.0).  
- If there is enough data → computes a smoothed effectiveness (alpha, beta).  
- Limits the value between 0.75 and 1.25 to avoid extremes.  

The result is a matrix that indicates which type tends to win against which in the dataset, later used for features such as `type_advantage`.

#### summarize_timeline()

Summarizes the entire sequence of the battle:

- Number of turns  
- Pokémon switches  
- Status changes  
- Damage received per turn  
- HP advantage per player  
- Momentum and control shifts  
- Whether there was a comeback  

It converts the battle’s progression into numerical variables that help the model understand how the battle was won.

#### team_static_features()

Extracts team statistics for each player:

- Means, maxima, minima, and standard deviations of base stats  
- Number of distinct team types  
- Type entropy (diversity)  

Returns all these features with a `p1_` or `p2_` prefix.

#### create_features_from_battle()

Combines all components:

- Team stats  
- Type advantages (player vs opponent)  
- Attribute differences  
- Battle timeline summary  
- Initial and final advantage  

Produces a dictionary with all the features of a battle.

#### build_feature_df()

Applies `create_features_from_battle()` to each battle in the dataset and builds the final DataFrame used to train the models.


In [12]:
from model_random import run_random
from model_stacking import run_stacking
from model_voting import run_voting


### Model `Random Forest`
After generating all the features and removing constant columns, we train a Random Forest Classifier, a model based on many decision trees that vote together.

We use stratified cross-validation to evaluate its performance in a stable way and avoid overfitting.
Finally, we train the full model with all available data and predict the outcomes of the battles in the test set.

The goal of this model is to provide a solid baseline against which the more advanced models (Voting and Stacking) can be compared.

In [13]:
run_random()


Running Random model...


Features: 100%|██████████| 5000/5000 [00:03<00:00, 1421.77it/s]


Removing constant columns: ['diff_base_hp', 'diff_base_atk', 'diff_base_def', 'diff_base_spa', 'diff_base_spd', 'diff_base_spe', 'n_turns']

Fold 1
Fold 1 accuracy: 0.8125

Fold 2
Fold 2 accuracy: 0.8185

Fold 3
Fold 3 accuracy: 0.8175

Fold 4
Fold 4 accuracy: 0.8085

Fold 5
Fold 5 accuracy: 0.8125

OOF Accuracy: 0.8139

CSV generated: submission_random.csv


### Model `Voting`

This model combines two different algorithms — XGBoost and AdaBoost — using soft voting.
Each model computes its probability of winning, and the VotingClassifier produces a final prediction by averaging both.

The idea is that XGBoost provides the ability to model complex interactions, while AdaBoost offers robustness and simplicity.
By combining them, the ensemble tends to be more stable and generalizes better than either model on its own.

This model also uses cross-validation to measure its performance, and then it is trained on all the data to generate the final prediction.

In [14]:
run_voting()


Loading data
Train: 10000 | Test: 5000

Cross validation (VotingClassifier: XGB + AdaBoost)...

Fold 1


Features: 100%|██████████| 2000/2000 [00:01<00:00, 1466.61it/s]


Fold 1 accuracy: 0.8190

Fold 2


Features: 100%|██████████| 2000/2000 [00:01<00:00, 1459.41it/s]


Fold 2 accuracy: 0.8285

Fold 3


...

Features: 100%|██████████| 2000/2000 [00:01<00:00, 1397.65it/s]


Fold 4 accuracy: 0.8220

Fold 5


Features: 100%|██████████| 2000/2000 [00:01<00:00, 1387.76it/s]


Fold 5 accuracy: 0.8255

Training the final model and generating the submission...


Features: 100%|██████████| 5000/5000 [00:03<00:00, 1436.55it/s]


### Model `Stacking`

This model combines three different algorithms (XGBoost, AdaBoost, and Gradient Boosting) using stacking.
Instead of voting, each model generates its predictions, and a final meta-model (Logistic Regression) learns how to combine them to improve accuracy.

Each model contributes a different perspective:

The system is trained with cross-validation to ensure robustness and is then fitted on all the data before generating the final prediction.

In [ ]:
run_stacking()
